In [ ]:
#importing kaggle
!pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! ls /content/drive


In [ ]:
!find /content/drive -name kaggle.json

In [ ]:
!ls /content/drive/MyDrive

In [ ]:
!ls /content

In [ ]:
#locating the dataset
!mkdir -p ~/.kaggle
!cp "/content/drive/MyDrive/MyDrive kaggle/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#importing the dataset
!kaggle datasets download -d pratyushakar/rossmann-store-sales

In [ ]:
#extracting the zip file
from zipfile import ZipFile
dataset="/content/rossmann-store-sales.zip"
with ZipFile(dataset,"r") as zip:
  zip.extractall()
  print("successfull")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
sales_dataset=pd.read_csv("/content/train.csv")

In [ ]:
sales_dataset.head()

In [ ]:
sales_dataset.shape

In [ ]:
sales_dataset.isnull().sum()

In [ ]:
store_dataset=pd.read_csv("/content/store.csv")

In [ ]:
store_dataset.head()

In [ ]:
store_dataset.shape

In [ ]:
store_dataset.isnull().sum()

In [ ]:
sns.displot(store_dataset["CompetitionDistance"])

In [ ]:
store_dataset.fillna({"CompetitonDistance":store_dataset["CompetitionDistance"].mode()},inplace=True)

In [ ]:
sns.displot(store_dataset["CompetitionOpenSinceMonth"])

In [ ]:
store_dataset.fillna({"CompetitionOpenSinceMonth":store_dataset['CompetitionOpenSinceMonth'].mean},inplace=True)

In [ ]:
sns.displot(store_dataset['CompetitionOpenSinceYear'])

In [ ]:
store_dataset.fillna({"CompetitionOpenSinceYear":store_dataset["CompetitionOpenSinceYear"].median()},inplace=True)

In [ ]:
sns.displot(store_dataset['Promo2SinceWeek'])

In [ ]:
store_dataset.fillna({"Promo2SinceWeek":store_dataset["Promo2SinceWeek"].mean()},inplace=True)

In [ ]:
sns.displot(store_dataset['Promo2SinceYear'])

In [ ]:
store_dataset.fillna({"Promo2SinceYear":store_dataset["Promo2SinceYear"].mean()},inplace=True)

In [ ]:
sns.displot(store_dataset['PromoInterval'])

In [ ]:
store_dataset.fillna({"PromoInterval":"unknown"},inplace=True)

In [ ]:
test_dataset=pd.read_csv("/content/test.csv")

In [ ]:
test_dataset.head()

In [ ]:
test_dataset.isnull().sum()

In [ ]:
sns.displot(test_dataset["Open"])

In [ ]:
test_dataset.head()

In [ ]:
test_dataset.fillna({"Open":test_dataset["Open"].mode()[0]},inplace=True)

In [ ]:
test_dataset.isnull().sum()

In [ ]:
sales_dataset["Date"]=pd.to_datetime(sales_dataset["Date"])
sales_dataset=sales_dataset.sort_values("Date")

In [ ]:
sales_dataset["month"]=sales_dataset["Date"].dt.month

In [ ]:
sales_dataset["Year"]=sales_dataset["Date"].dt.year

In [ ]:
sales_dataset=sales_dataset.set_index("Date")

In [ ]:
sales_monthly=sales_dataset["Sales"].resample("ME").sum()

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(sales_monthly.index,sales_monthly.values)
plt.title("Monthly Sales")
plt.xlabel("Date")
plt.ylabel("Sum of Sales")
plt.plot()

In [ ]:
sales_week=sales_dataset["Sales"].resample("W").sum()

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(sales_week.index,sales_week.values)
plt.title("WEEKLY SALES")
plt.xlabel("Date")
plt.ylabel("Sum of Sales")
plt.plot()

In [ ]:
sales_dataset["Isholiday"]=sales_dataset["SchoolHoliday"].apply(lambda x:"yes" if x==1 else "working")

In [ ]:
sales_dataset["sales_lag"]=sales_dataset["Sales"].shift(1)
sales_dataset["rolling_30"]=sales_dataset["Sales"].rolling(30).mean()

In [ ]:
train_size=int(len(sales_dataset)*0.8)
train=sales_dataset.iloc[:train_size]
test=sales_dataset[train_size:]

In [ ]:
from os import name
state_holidays=sales_dataset[sales_dataset["StateHoliday"]!=0].index.to_frame(name="ds")
state_holidays["holiday"]="stateholiday"

In [ ]:
state_holidays.reset_index(drop=True)

In [ ]:
from os import name
school_holidays=sales_dataset[sales_dataset['SchoolHoliday']==1].index.to_frame(name="ds")
school_holidays["holiday"]="schoolholidays"

In [ ]:
school_holidays.reset_index(drop=True)

In [ ]:
holiday_df=pd.concat([state_holidays,school_holidays])
holiday_df["lower_window"]=0
holiday_df["upper_window"]=1
holiday_df=holiday_df.drop_duplicates()

In [ ]:
from prophet import Prophet
model_1=Prophet(holidays=holiday_df)
model_1.add_regressor("Customers")
model_1.add_regressor("Promo")

In [ ]:
#dataframe
df_prophet=sales_dataset.copy()
df_prophet["ds"]=df_prophet.index
df_prophet=df_prophet[["ds","Sales","Customers","Promo"]]
df_prophet=df_prophet.rename(columns={"Sales":"y"})

In [ ]:
print(df_prophet.columns)

In [ ]:
#train the model
model_1.fit(df_prophet)

In [ ]:
#forecasting
future=model_1.make_future_dataframe(periods=30)
future["Customers"]=df_prophet["Customers"].mean()
future["Promo"]=0
forecast=model_1.predict(future)

In [ ]:
print(forecast)

In [ ]:
test.head()

In [ ]:
test.head()

In [ ]:
ytrue=test["Sales"].values
ypred=forecast.set_index("ds").loc[test.index,"yhat"].values
from sklearn.metrics import mean_absolute_error
#for actual,predicted in zip(ytrue,ypred):
mae=mean_absolute_error(ytrue,ypred)
print("MAE:",mae)


In [ ]:
sales_dataset.tail()

In [ ]:
req_df=pd.DataFrame({"Date":test.index,"Actual_sales":ytrue,"forecasted_sales":ypred,"Store":test["Store"],"Week":test["DayOfWeek"],
                     "Month":test["month"],"Year":test["Year"],"stateholiday":test["StateHoliday"],"schoolholiday":test["SchoolHoliday"],
                     "holiday":test["Isholiday"],"forecasted_error":ytrue-ypred})


In [ ]:
req_df.to_csv("forecasted_sales.csv",index=False)

In [ ]:
from google.colab import files
files.download("forecasted_sales.csv")